In [1]:
import numpy as np
import pandas as pd
pd.set_option('chained_assignment',None)

In [2]:
cat = ["50-59 months","60-69 months"]

In [3]:
def categorize(df):
    df=df[df.child_age_group.isin(cat)]
    # initializee category
    df["category"] = " "
    # 50-59 months
    df["category"][(df.child_age_group=="50-59 months")&(df.target<=36.01)]="Red"
    df["category"][(df.child_age_group=="50-59 months")&(df.target>36.01)&(df.target<=46.31)]="Yellow"
    df["category"][(df.child_age_group=="50-59 months")&(df.target>46.31)&(df.target<=100)]="Green"
    # 60-69 months
    df["category"][(df.child_age_group=="60-69 months")&(df.target<=43.23)]="Red"
    df["category"][(df.child_age_group=="60-69 months")&(df.target>43.23)&(df.target<=54.37)]="Yellow"
    df["category"][(df.child_age_group=="60-69 months")&(df.target>54.37)&(df.target<=100)]="Green"
    return df

In [4]:
fts = [
    "teacher_emotional_total",
    "count_children_present",
    "count_staff_all",
    "count_register_gender_male",
    "count_register_gender_female",
    "count_register_race_white",
    "count_register_race_coloured",
    "teacher_duration",
    "pqa_score_teaching",
    "child_months_enrolment",
    "child_observe_total",
    "child_zha",
    "child_attendance",
    "child_height",
    "child_age"
]

In [5]:
report = pd.DataFrame()

for feat in fts:
    first_elom = pd.read_csv("first/first_elom.csv")
    first = pd.read_csv("first/first.csv")[["child_id","child_age_group",feat]]
    first = pd.merge(first,first_elom,on="child_id",how="left")
    first = categorize(first)
    first=first[["child_id",feat,"target","category","feature_1","feature_2","feature_3"]]
    new_elom = pd.read_csv(f"{feat}_elom.csv")
    new = pd.read_csv(f"median_{feat}.csv")
    new = pd.merge(new,new_elom,on="child_id",how="left")
    new = categorize(new)
    new=new[["child_id",feat,"target","category","feature_1","feature_2","feature_3"]]
    first = first.iloc[:,:4].rename({feat:"current_variable","target":"current_elom","category":"current_category"},axis=1)
    new = new.iloc[:,:4].rename({feat:"new_variable","target":"new_elom","category":"new_category"},axis=1)
    df = pd.merge(first,new,on='child_id',how='left')
    df["relevant_category_change"] = df["current_category"] + " to " + df["new_category"]
    df = df[df.current_variable!=df.new_variable]
    df = df[df.current_category!=df.new_category]
    df = df.drop(["current_category","new_category"],axis=1)
    df["variable"] = feat
    df.to_csv(f"{feat}_sensitivity.csv",index=None)
    report = report.append(df)
report = report.dropna(axis=0)
report.to_csv("ALL_REPORTS.csv",index=None)
report

C:\Users\hp\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (172,252,254,257,275,343,358,359,461,641,642,643) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\hp\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (172,252,254,257,275,343,358,359,461,641,642,643) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\hp\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (172,252,254,257,275,343,358,359,461,641,642,643) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\hp\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (172,252,254,257,275,343,358,

,child_id,current_variable,current_elom,new_variable,new_elom,relevant_category_change,variable
1,ID_GQ6ONJ4FP,12.000000,47.383709,10.000000,44.197728,Green to Yellow,teacher_emotional_total
4,ID_1U7GDTLRI,7.000000,45.441186,10.000000,48.070224,Yellow to Green,teacher_emotional_total
9,ID_C9RZNXMYF,12.000000,49.704687,10.000000,45.029700,Green to Yellow,teacher_emotional_total
24,ID_WYP2V4JWK,12.000000,36.125227,10.000000,35.831965,Yellow to Red,teacher_emotional_total
80,ID_G8LXFG5SP,7.000000,45.045530,10.000000,46.809690,Yellow to Green,teacher_emotional_total
...,...,...,...,...,...,...,...
3628,ID_ARO2FW1J4,52.000000,35.501016,55.000000,38.714403,Red to Yellow,child_age
3646,ID_W9LBCFP8T,59.311474,46.518412,55.000000,44.018108,Green to Yellow,child_age
3657,ID_OLMUJZ817,50.000000,30.711558,55.000000,37.447765,Red to Yellow,child_age
3661,ID_YQ79K5YAB,52.000000,32.812336,55.000000,38.752916,Red to Yellow,child_age


In [6]:
report["variable"].value_counts()

child_observe_total             732
child_age                       357
teacher_emotional_total         294
child_height                    269
child_zha                       202
count_children_present          163
count_register_gender_male      149
count_staff_all                 141
count_register_gender_female    136
child_months_enrolment           79
teacher_duration                 64
count_register_race_coloured     47
count_register_race_white         6
Name: variable, dtype: int64